<h1>The Classical and Quantum Base Size</h1>

<h2>Permutation Groups in SAGE</h2>

Permutation groups can be described by generators, or by using some pre-named groups. We always assume a permutation group of degree $n$ acts on the particular set $\Omega = \{1, \dots, n\}$.

In [309]:
G = PermutationGroup(['(1,2)', '(2,3)(4,5)']); G

Permutation Group with generators [(2,3)(4,5), (1,2)]

In [310]:
SymmetricGroup(3)==G

False

In [297]:
G.stabilizer(3)

Subgroup generated by [(1,2)] of (Permutation Group with generators [(2,3), (1,2)])

<h2>minBase</h2>

A naive algorithm to compute the base size of a permutation group.
We compute all stabilizer subgroups for singletons, then find the smallest number of subgroups which
intersect trivially (using depth first search). Deciding whether the
base size is below a given constant is an NP-complete problem (Blaha).

Args: G permutation group object

Returns: a minimal base for $G$ (list of numbers in $\{1, \dots, n\}$).


In [41]:

def minBase(G):
    # Generate stabilizer subgroups:
    D = 1
    n = G.degree()
    stabs = {Set({x+1}) : G.stabilizer(x+1) for x in range(n)}
    while D < n:
        D = D+1
        for indices in Subsets(n, D): # iterate over all size D subsets of [1,..n]
            rand_singleton = Set({indices.random_element()}) # pick a random element to remove
            old_indices =  indices.difference(rand_singleton) # removing results in D-1 indices
            new_stab = stabs[old_indices].intersection(stabs[rand_singleton])
            stabs[indices] = new_stab
            
            if new_stab.cardinality() == 1:
                return list(indices)
        
    return list(range(1,n));
        

In [74]:
# On my computer, the algorithm finds a minimal base for S_11 with in appox 1 sec.
%time minBase(SymmetricGroup(11))

CPU times: user 906 ms, sys: 16 ms, total: 922 ms
Wall time: 916 ms


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

<h2>quantumBaseSize and bddErrorQuantumBaseSize</h2>

Computes exactly this, using character theory.

Args: G permutation group object

quantumBaseSize returns: 

$\quad$ the smallest $t$ such that $\Omega^{\otimes t}$ contains every irrep of $G$.

bddErrorQuantumBaseSize returns: 

$\quad$   the smallest $t$ such that $\mathbb{P}(\Omega^{\otimes t}) \geq 2/3$, where
$$
\mathbb{P}(\Omega^{\otimes t}) = \sum_{\chi}\chi(1)^2,
$$
with the sum taken over all irreducible characters $\chi$ which appear in $\Omega^{\otimes t}$. Note that $\chi(1)$ is equal to the <strong>degree</strong> of $\chi$, i.e. the dimension of the irrep corresponding to $\chi$.

In [301]:
'''
permChar

Args: G permutation group object
Returns: the character of the permutation action

'''

def permChar(G):
    n = G.degree()
    conjClassReps = G.conjugacy_classes_representatives()
    values = [n - sum(list(g.cycle_type(singletons=False))) for g in conjClassReps]
    return ClassFunction(G, values)

'''
dimsSquared(G,V)

Args: G permutation group object
    V character of G (ClassFunction object)


'''
def dimsSquared(G, V, irr=G.irreducible_characters()):
    dims = 0
    for chi in irr:
        chi = ClassFunction(G, chi)
        if chi.scalar_product(V) != 0:
            dims = dims + list(chi)[0]^2
    return dims


def quantumBaseSize(G):
    n = G.degree()
    M = G.order()
    irr = G.irreducible_characters()
    irrs_collected = []
    irrs_new = []
    irrs_missing = irr
    dims = 0
    prob = 0
    
    V = permChar(G)
    Vt = V
    t = 1
    while t < n:
        for chi in irrs_missing:
            chiF = ClassFunction(G, chi)
            
            if (chiF.scalar_product(Vt) != 0):
                irrs_new.append(chi)
                dims = dims + list(chiF)[0]^2
                prob = prob + list(chiF)[0]^2/M
                
        irrs_missing = [x for x in irrs_missing if x not in irrs_new]        
        
        print(str(t) + ' query success prob = ' + str(prob) + ' ~ ' + str(float(prob)))
        if dims == M:
            print("     Quantum base size = " + str(t))
            return t
        else:
            Vt = V*Vt
            t = t+1
    return t

def bddErrorQuantumBaseSize(G):
    n = G.degree()
    M = G.order()
    irr = G.irreducible_characters()
    irrs_collected = []
    irrs_new = []
    irrs_missing = irr
    dims = 0
    prob = 0
    
    V = permChar(G)
    Vt = V
    t = 1
    while t < n:
        for chi in irrs_missing:
            chiF = ClassFunction(G, chi)
            
            if (chiF.scalar_product(Vt) != 0):
                irrs_new.append(chi)
                dims = dims + list(chiF)[0]^2
                prob = prob + list(chiF)[0]^2/M
        irrs_missing = [x for x in irrs_missing if x not in irrs_new]        
        
        print(str(t) + ' query success prob = ' + str(prob) + ' ~ ' + str(float(prob)))
        if float(prob) > 2/3:
            print("     Quantum base size = " + str(t))
            return t
        else:
            Vt = V*Vt
            t = t+1
    return t

<h2>Examples</h2>

In [302]:
quantumBaseSize(MathieuGroup(10)); minBase(MathieuGroup(10))

1 query success prob = 41/360 ~ 0.11388888888888889
2 query success prob = 719/720 ~ 0.9986111111111111
3 query success prob = 1 ~ 1.0
     Quantum base size = 3


[1, 2, 3]

In [303]:
# The quantumBaseSize method starts failing... not sure why.

quantumBaseSize(MathieuGroup(11)); minBase(MathieuGroup(11))

1 query success prob = 1 ~ 1.0
     Quantum base size = 1


[1, 2, 3, 4]

<h3>Using the outer automorphism of $S_6$.</h3>

We consider $G \cong S_6$ acting on a set of order 12, with 2 orbits. The first orbit (consisting of $\{1, \dots, 6\}$ is just the usual defining action. The second orbit ($\{7, \dots, 12\})$ is the defining action, twisted by the non-trivial outer automorphism of $S_6$.

In [312]:


G = PermutationGroup(['(1,2)(7,8)(9,10)(11,12)', '(1,2,3,4,5,6)(8,10)(9,12,11)']); G.order()

720

In [313]:
# Classically we need at least 4 guesses
minBase(G)

[1, 2, 3, 7]

In [314]:
# We can get 93% success with just 2 quantum queries! Still need 4 for exact learning.

quantumBaseSize(G)

1 query success prob = 17/240 ~ 0.07083333333333333
2 query success prob = 223/240 ~ 0.9291666666666667
3 query success prob = 719/720 ~ 0.9986111111111111
4 query success prob = 1 ~ 1.0
     Quantum base size = 4


4

In [276]:
%time quantumBaseSize(PermutationGroup([(1,2), (2,3), (3,4,6)]))

1 query success prob = 17/120 ~ 0.14166666666666666
2 query success prob = 13/20 ~ 0.65
3 query success prob = 119/120 ~ 0.9916666666666667
4 query success prob = 1 ~ 1.0
     Quantum base size = 4
CPU times: user 3.23 s, sys: 484 ms, total: 3.72 s
Wall time: 3.73 s


4